# SIR Dynamic
# Data-Import

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_rows',500)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import optimize
from scipy import integrate
import plotly

import plotly.express as px
import plotly.graph_objects as go

sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

In [2]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

time_idx=pd_raw.columns[4:]
df_new=pd.DataFrame({'date':time_idx})

country=['Germany','India','US']

for each in country:
    df_new[each]=np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4:].sum(axis=0))

df_new['date']=df_new.date.astype('datetime64[ns]')

df_new.to_csv('../data/processed/small_flat_table.csv',sep=';',index=False)

In [ ]:
df_new

# General Functions

In [4]:
def SIR_model(SIR,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return([dS_dt,dI_dt,dR_dt])

In [5]:
def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

In [6]:
def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

# Loop for all country

In [ ]:
ydata = np.array(df_new['Germany'][35:100])
t=np.arange(len(ydata))
x=np.linspace(1,len(ydata),len(ydata))
df_plot=pd.DataFrame({'day':np.array(x)})

for each in country: 
    
    if each=='Germany':
        N0=80000000 #max susceptible population
    elif each=='India':
        N0=1380000000
    else:
        N0=329000000
        
    beta=0.4   # infection spread dynamics
    gamma=0.1
    
    I0=df_new[each][35]
    S0=N0-I0
    R0=0
    
    ydata = np.array(df_new[each][35:100])
    t=np.arange(len(ydata))
    
    # example curve of our differential equationa
    popt=[beta,gamma]
    fit_odeint(t, *popt);
    
    popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
    perr = np.sqrt(np.diag(pcov))
    
    print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
    print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
    
    # get the final fitted curve
    fitted=fit_odeint(t, *popt)
    len(fitted)
    x=np.linspace(1,len(fitted),len(fitted))
    #df_plot=pd.DataFrame({'day':np.array(x)})
    new={each+'_fit':np.array(fitted),
         each+'_case':np.array(df_new[each][35:100])
        }
    pd_new=pd.DataFrame(new)
    df_plot=pd.concat([df_plot,pd_new],axis=1)
    print(each+":Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
    print(each+"Basic Reproduction Number R0 " , popt[0]/ popt[1])

In [ ]:
df_plot

# Visualization

In [ ]:
fig=go.Figure()
for each in country:
    fig.add_trace(go.Scatter(x=df_plot.day,
                             y=df_plot[each+'_case'],
                             mode='markers',
                             marker_size=8,
                             name=each+'_case'
                         ))
for each in country:
    fig.add_trace(go.Scatter(x=df_plot.day,
                             y=df_plot[each+'_fit'],
                             mode='lines',
                             name=each+'_fit'
                         ))

fig.update_layout(
        width=1000,
        height=800,
        xaxis_title="Days",
        yaxis_title="Covid infections",
)
fig.update_yaxes(type="log")
fig.show()

# Dash

In [10]:
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import os
print(os.getcwd())

fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Goal of the project is to demonstrate the SIR prediction model for three countries and the reproduction rate,
    infection rate and recovery rate for the first few days of the pandemic

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),
    
    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in country],
        value=['US', 'Germany','India'], # which are pre-selected
        multi=True
    ),

 

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])


def update_figure(country):


    traces = []
    for each in country:
        traces.append(dict(x=df_plot.day,
                                y=df_plot[each+'_case'],
                                mode='markers',
                                marker_size=8,
                                name=each+'_case'
                       )
                )
    
    for each in country:
        traces.append(dict(x=df_plot.day,
                                y=df_plot[each+'_fit'],
                                mode='lines',
                                name=each+'_fit'
                       )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis={'type':"log",
                       'range':'[1.1,5.5]'
                      }
        )
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False,port=8051)